In [14]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, PILToTensor
import time
import csv
torch.set_num_threads(1)


In [15]:
# Example of target with class indices
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
print(input)
print(target)
output = loss(input, target)
output.backward()
# Example of target with class probabilities
input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5).softmax(dim=1)
print(input)
print(target)
output = loss(input, target)
output.backward()

tensor([[-1.1160, -0.7728,  1.1024,  1.4356, -0.4694],
        [ 0.7554,  0.2034, -0.4377, -0.0049, -1.8858],
        [-0.0511, -1.2432,  0.5693,  0.7769, -1.4020]], requires_grad=True)
tensor([0, 1, 3])
tensor([[-0.8704,  0.7439, -1.0992,  1.1597, -1.0936],
        [ 1.6606, -0.7092, -0.2288,  0.4138, -1.3268],
        [ 1.7225, -1.0450,  1.3364,  0.0473,  0.0850]], requires_grad=True)
tensor([[0.0526, 0.5039, 0.0432, 0.0360, 0.3643],
        [0.3617, 0.1378, 0.0299, 0.1484, 0.3222],
        [0.0399, 0.0920, 0.2653, 0.4795, 0.1234]])


In [16]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cpu device


In [17]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
    # transform=PILToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [18]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [19]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [20]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [21]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([8])


In [22]:
loss_fn = nn.CrossEntropyLoss()
# loss_fn = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [23]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X.float())
        # print("pred: ",pred.shape)
        # print("y:",y.shape)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [24]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return correct

In [25]:
epochs = 3
for datasize in [7500,15000,30000,60000]:
    for epoch in range(epochs):
        subset_training_data = Subset(training_data, range(datasize))
        train_dataloader_subset = DataLoader(subset_training_data, batch_size=batch_size)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch} took {epoch_duration} seconds")
        with open("datasize_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, batch_size, datasize, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 2.305422  [   64/ 7500]
loss: 2.287650  [ 6464/ 7500]
Test Error: 
 Accuracy: 14.2%, Avg loss: 2.288500 

Epoch 0 took 0.8753582879435271 seconds
Epoch 2
-------------------------------
loss: 2.290825  [   64/ 7500]
loss: 2.274626  [ 6464/ 7500]
Test Error: 
 Accuracy: 17.2%, Avg loss: 2.272207 

Epoch 1 took 0.8716882839798927 seconds
Epoch 3
-------------------------------
loss: 2.276398  [   64/ 7500]
loss: 2.261150  [ 6464/ 7500]
Test Error: 
 Accuracy: 21.3%, Avg loss: 2.255655 

Epoch 2 took 0.8708253629738465 seconds
Epoch 1
-------------------------------
loss: 2.261758  [   64/15000]
loss: 2.247007  [ 6464/15000]
loss: 2.220557  [12864/15000]
Test Error: 
 Accuracy: 37.8%, Avg loss: 2.220372 

Epoch 0 took 1.741135093034245 seconds
Epoch 2
-------------------------------
loss: 2.230200  [   64/15000]
loss: 2.215467  [ 6464/15000]
loss: 2.179714  [12864/15000]
Test Error: 
 Accuracy: 45.4%, Avg loss: 2.180484 

Epoch 1 took 1.739863

In [26]:
epochs = 3
for bsize in [32,64,128,256]:
    for epoch in range(epochs):
        train_dataloader_subset = DataLoader(training_data, batch_size=bsize)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch} took {epoch_duration} seconds")
        with open("minibatch_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, bsize, 60000, accuracy, test_duration])

print("Done!")

Epoch 1
-------------------------------
loss: 1.009737  [   32/60000]
loss: 1.038983  [ 3232/60000]
loss: 1.039360  [ 6432/60000]
loss: 1.155540  [ 9632/60000]
loss: 0.942007  [12832/60000]
loss: 1.133674  [16032/60000]
loss: 0.842636  [19232/60000]
loss: 0.867481  [22432/60000]
loss: 0.939195  [25632/60000]
loss: 1.075358  [28832/60000]
loss: 0.887192  [32032/60000]
loss: 0.821210  [35232/60000]
loss: 1.148261  [38432/60000]
loss: 0.975416  [41632/60000]
loss: 0.921393  [44832/60000]
loss: 0.809415  [48032/60000]
loss: 0.912361  [51232/60000]
loss: 1.084087  [54432/60000]
loss: 0.817860  [57632/60000]
Test Error: 
 Accuracy: 67.5%, Avg loss: 0.895853 

Epoch 0 took 7.7205545720644295 seconds
Epoch 2
-------------------------------
loss: 0.827731  [   32/60000]
loss: 0.904917  [ 3232/60000]
loss: 0.885419  [ 6432/60000]
loss: 1.036878  [ 9632/60000]
loss: 0.786566  [12832/60000]
loss: 1.013987  [16032/60000]
loss: 0.700844  [19232/60000]
loss: 0.714831  [22432/60000]
loss: 0.804677  [2

In [ ]:
epochs = 3
for bsize in [32,64,128,256]:
    for epoch in range(epochs):
        train_dataloader_subset = DataLoader(training_data, batch_size=bsize)
        print(f"Epoch {epoch+1}\n-------------------------------")
        epoch_time_start = time.perf_counter()
        train(train_dataloader_subset, model, loss_fn, optimizer)
        epoch_time_end = time.perf_counter()

        test_start_time = epoch_time_end
        accuracy = test(test_dataloader, model, loss_fn)
        test_end_time = time.perf_counter()

        epoch_duration = epoch_time_end - epoch_time_start
        test_duration = test_end_time - test_start_time

        print(f"Epoch {epoch} took {epoch_duration} seconds")
        with open("minibatch_nn.csv", "a") as fp:
            wr = csv.writer(fp, dialect='excel')
            # epoch_duration, epoch, batch_size, data_size, accuracy, test_duration
            wr.writerow([epoch_duration, epoch, bsize, 60000, accuracy, test_duration])

print("Done!")

In [27]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [28]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>